In [3]:
# leafspine
numserver = 3072
numsw = 80
numport = 64 # == numleaf

file = f"/home/annzhou/DRing/src/emp/datacentre/serverfiles/leafspine_{numserver}_{numsw}_{numport}"
numserverpersw = numserver//numport
with open(file,'w') as f:
    for server in range(numserver):
        sw = server//numserverpersw
        f.write(f"{server},{sw}\n")

In [11]:
# rrg,dring -- this is wrong, at least for DRing_2988
# inspired by int RandRegularTopology::ConvertHostToSwitch(int host)
numserver = 2988
numsw = 80
numport = 64 # == numleaf
graphname = "dring"

def server_to_sw(host):
    OVERSUBSCRIPTION = 1
    NSW = numsw
    NHOST = numserver
    REDUCED_NHOST = NHOST/OVERSUBSCRIPTION
    SVRPORTS = OVERSUBSCRIPTION * (REDUCED_NHOST/NSW if REDUCED_NHOST%NSW==0 else REDUCED_NHOST/NSW+1)
    HETERO = 0 if REDUCED_NHOST%NSW==0 else 1
    nsw_less_svrport = NSW - (NHOST/OVERSUBSCRIPTION)%NSW

    if HETERO==1:
        if SVRPORTS!=1:
            reducedSvrport = SVRPORTS - OVERSUBSCRIPTION
            if host < nsw_less_svrport * reducedSvrport:
                myswitchindex = host // reducedSvrport
            else:
                myswitchindex = (host - nsw_less_svrport * reducedSvrport) // (SVRPORTS) + nsw_less_svrport
        else:
            myswitchindex =  nsw_less_svrport + host
    else:
        myswitchindex = host // SVRPORTS

    return int(myswitchindex)

file = f"/home/annzhou/DRing/src/emp/datacentre/serverfiles/{graphname}_{numserver}_{numsw}_{numport}"
with open(file,'w') as f:
    for server in range(numserver):
        sw = server_to_sw(server)
        f.write(f"{server},{sw}\n")

fix for dring_2988

In [12]:
txtfile = f"/home/annzhou/DRing/src/emp/datacentre/makeserverfiles/dring_2988_fromlogging"
serverfile = f"/home/annzhou/DRing/src/emp/datacentre/serverfiles/dring_2988_80_64"
with open(txtfile,'r') as fr:
    lines = fr.readlines()
    with open(serverfile,'w') as fw:
        prevsw = -1
        prevserver = -1
        baseserver = 0
        for line in lines:
            if line.startswith("SVR"): continue
            tokens = line.split("->")
            sw = int(tokens[0][2:])
            server = int(tokens[1][3:])
            # print(f"sw={sw},prevsw={prevsw}")
            if sw != prevsw:
                baseserver = prevserver+1
                # print(baseserver)
            prevsw = sw
            prevserver = server+baseserver
            fw.write(f"{server+baseserver},{sw}\n")